##**Dataset 1 : COCO128**



**SECTION 0 – Setup**

In [8]:

import sys
#sys.path.append('Projet_ML/od-project/src')
sys.path.append('Projet_ML/od-project')


# === Check GPU ===
!nvidia-smi

# === Install Dependencies ===
!pip install -r requirements.txt

# === Imports + Seeds ===
import torch, numpy as np, random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("Device:", "cuda" if torch.cuda.is_available() else "cpu")

Mon Nov 24 20:19:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
print("GPU disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU")

GPU disponible : True
Nom du GPU : Tesla T4


In [6]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.to('cuda')
print("✅ YOLO est prêt")
print("Type de modèle :", type(model.model).__name__)
print("Device utilisé :", model.device)

ModuleNotFoundError: No module named 'ultralytics'

**SECTION 1 – Problem Definition**

Objectif du projet : "Développer un modèle de détection d’objets sur le dataset COCO128"

Tâche ML : “Object Detection”

Modèle choisi : YOLOv8n (Ultralytics)

Dataset mentionné (COCO128 ou ton dataset)

Métriques utilisées : mAP@0.5, précision, rappel

Contexte métier (autonomous vehicle / drone / inventaire)

Pourquoi c’est un bon choix pour ce problème

**SECTION 2 – Data Validation**

Nom du dataset (COCO128 ou ton dataset custom)

Source : Ultralytics / Kaggle / Google Drive

Nombre d’images / classes

Structure YOLO : /images/train, /images/val, /labels/...

Format des labels : class_id x_center y_center width height

Licence, biais éventuels

In [ ]:
#from PIL import Image
#Image.open("ultralytics/assets/bus.jpg")

In [ ]:
from ultralytics import YOLO
import time, json, os

# 1. Charger modèle
model = YOLO("yolov8n.pt")
model.to("cuda")  # GPU garanti

# 2. Prédiction rapide
print("Running smoke check on GPU...")
t0 = time.time()
results = model.predict(source="https://ultralytics.com/images/bus.jpg", imgsz=640)
elapsed = time.time() - t0

# 3. Sauvegarde des résultats
os.makedirs("outputs", exist_ok=True)
metrics = {
    "model": "yolov8n.pt",
    "device": str(model.device),
    "success": True,
    "elapsed_time": round(elapsed, 2)
}
with open("outputs/smoke_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print("\n✅ Smoke check terminé avec succès !")
print(metrics)

**SECTION 3 – Baseline Training**

In [ ]:
from train import main

main("configs/yolo_coco128.yaml")

hyperparamètres (epochs, lr, batch, imgsz)

pertes observées, première mAP obtenue

**SECTION 4 – Evaluation & Prediction**

In [ ]:
from evaluate import main as eval_main

cfg = "configs/yolo_coco128.yaml"
ckpt = "outputs/train/weights/best.pt"

eval_main(cfg, ckpt)

In [ ]:
from predict import main as predict_main

cfg = "configs/yolo_coco128.yaml"
ckpt = "outputs/train/weights/best.pt"
img = "https://ultralytics.com/images/bus.jpg"  # image valide

predict_main(cfg, ckpt, img)

In [ ]:
from PIL import Image
Image.open("outputs/pred/vis/bus.jpg")

Analyse des résultats : quelles classes sont bien détectées, quelles sont ratées ?

**SECTION 5 – Ablation Studies**

In [ ]:
from ultralytics import YOLO
from utils import load_yaml
import json
from pathlib import Path

def run_experiment(config_path, exp_name, override_params=None):
    """
    Lance un entraînement YOLO avec modifications d'hyperparamètres.

    - config_path : chemin vers yolo_coco128.yaml
    - exp_name : nom de l'expérience ("lr_small", "epochs_10", etc.)
    - override_params : dict { "lr0": 0.005, "epochs": 10, ... }
    """
    cfg = load_yaml(config_path)
    override_params = override_params or {}

    # appliquer les modifications
    cfg["train"].update(override_params)

    print(f"🔧 Lancement de l'expérience : {exp_name}")
    print("Hyperparamètres modifiés :", override_params)

    model = YOLO(cfg["model"]["weights"])
    results = model.train(
        data=cfg["data"]["yaml"],
        imgsz=cfg["data"]["imgsz"],
        epochs=cfg["train"]["epochs"],
        batch=cfg["data"]["batch"],
        lr0=cfg["train"]["lr0"],
        weight_decay=cfg["train"]["weight_decay"],
        patience=cfg["train"]["patience"],
        device=cfg.get("device", "auto"),
        project="outputs/ablations",
        name=exp_name,
        verbose=False
    )

    # charger meilleur modèle
    best = Path(f"outputs/ablations/{exp_name}/weights/best.pt")
    model = YOLO(str(best))

    # évaluation
    val = model.val(data=cfg["data"]["yaml"], imgsz=cfg["data"]["imgsz"], verbose=False)

    metrics = {
        "experiment": exp_name,
        "lr0": cfg["train"]["lr0"],
        "epochs": cfg["train"]["epochs"],
        "map50": val.box.map50,
        "map50_95": val.box.map,
        "precision": val.box.mp,
        "recall": val.box.mr,
    }

    return metrics



In [ ]:
exp1 = run_experiment(
    "configs/yolo_coco128.yaml",
    exp_name="exp_lr_small",
    override_params={"lr0": 0.005}
)
exp1

In [ ]:
exp2 = run_experiment(
    "configs/yolo_coco128.yaml",
    exp_name="exp_epochs10",
    override_params={"epochs": 10}
)
exp2

In [ ]:
import json

# Charger les métriques de la baseline
with open("outputs/metrics.json", "r") as f:
    baseline_metrics = json.load(f)
baseline_metrics

In [ ]:
baseline = {
    "experiment": "baseline",
    "lr0": 0.01,      # valeur par défaut dans ton YAML
    "epochs": 5,      # valeur par défaut
    "map50": baseline_metrics["metrics"]["map50"],
    "map50_95": baseline_metrics["metrics"]["map"],
    "precision": baseline_metrics["metrics"]["precision"],
    "recall": baseline_metrics["metrics"]["recall"],
}
baseline

In [ ]:
import pandas as pd

df = pd.DataFrame([baseline, exp1, exp2])
df




In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def show_image(path, title=""):
    try:
        img = Image.open(path)
        plt.figure(figsize=(6,6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.show()
    except FileNotFoundError:
        print(f"⚠️ Fichier introuvable : {path}")

In [ ]:
show_image("outputs/train/confusion_matrix_normalized.png", "Confusion Matrix — Baseline")

show_image("outputs/ablations/exp_lr_small/confusion_matrix_normalized.png", "Confusion Matrix — lr0 plus petit")

show_image("outputs/ablations/exp_epochs10/confusion_matrix_normalized.png", "Confusion Matrix — 10 epochs")


Commentaires : pourquoi l’un est meilleur ?

Que conclure sur la sensibilité du modèle ?

**SECTION 6 – Final Results & Conclusion**

In [ ]:
import json

with open("outputs/eval.json", "r") as f:
    metrics_final = json.load(f)

metrics_final

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def show_img(path, title=None):
    img = Image.open(path)
    plt.figure(figsize=(8,6))
    plt.imshow(img)
    plt.axis("off")
    if title:
        plt.title(title)
    plt.show()

show_img("outputs/train/val_batch0_pred.jpg", "Exemple de détection finale")

##**Conclusion**

Dans ce projet, nous avons entraîné un modèle YOLOv8n sur le dataset COCO128.
Malgré un nombre d’époques réduit (5) et la taille limitée du dataset, le modèle parvient à atteindre une performance correcte avec un mAP50 de 0.65, ce qui montre sa capacité à généraliser même dans des conditions d’entraînement limitées.

Les hyperparamètres utilisés sont :

imgsz = 640

batch = 16

epochs = 5

lr0 = 0.01

weight_decay = 0.0005

device = GPU (Tesla T4 via Google Colab)

Le meilleur modèle obtenu est sauvegardé dans :outputs/train/weights/best.pt

**Performance globale**

mAP50 ≈ 0.65

mAP50–95 ≈ 0.48

Précision ≈ 0.67

Recall ≈ 0.58

Pour un modèle nano, 5 epochs, et un mini-dataset, ces scores sont totalement cohérents.

Les classes les plus simples (personne, bus, animaux visibles) sont détectées avec une grande fiabilité.
En revanche, les objets petits, peu représentés, ou fortement occlus présentent des performances faibles.
La confusion matrix met également en évidence des confusions logiques entre objets visuellement proches (car ↔ truck, handbag ↔ person).


**Fausses détections (FP)**

On observe dans la confusion matrix :

plusieurs objets confondus avec person

erreurs sur car ↔ truck

erreurs petites classes (mouse, cell phone) ⟶ souvent non détectées

Cela est lié à la limitation naturelle de YOLOv8n (très petit modèle + 5 epochs)

**Expérience 1 :**

Dans la première expérience, nous avons réduit le learning rate initial.
Cette modification a deux conséquences principales :

Le modèle apprend plus lentement, ce qui se traduit par une légère baisse des métriques globales (mAP et précision).

Le rappel augmente légèrement, signe que le modèle manque moins d’objets, mais qu'il produit davantage de fausses détections.

En résumé, baisser le learning rate rend l’apprentissage plus prudent, ce qui peut être utile si on augmente le nombre d’epochs, mais dans notre cas, avec seulement 5 epochs, cela donne un modèle globalement moins performant que le baseline.


**Expérience 2:**

Dans la troisième expérience, nous avons simplement doublé le nombre d’epochs, ce qui signifie que le modèle a vu le dataset deux fois plus longtemps.

Cette modification améliore clairement les performances :

Le mAP50 et le mAP50-95 augmentent.

Le rappel s’améliore également.

La précision reste globalement similaire à celle du modèle de base.

Ces résultats montrent que le modèle n’était pas encore à convergence après 5 epochs, ce qui est logique sur COCO128. Avec plus d’epochs, le modèle apprend davantage de détails, ce qui améliore la qualité générale des prédictions.


**Conclusion générale**

Les expériences montrent que les hyperparamètres influencent fortement la qualité d’un modèle YOLO :

Réduire le learning rate sans augmenter les epochs ralentit trop l’apprentissage et dégrade les performances globales.

Augmenter le nombre d’epochs est un moyen simple et efficace d’améliorer la performance, surtout sur un petit dataset comme COCO128.

Globalement, le modèle entraîné pendant 10 epochs offre les meilleurs résultats parmi les trois, confirmant qu’un entraînement plus long permet au réseau de mieux converger.

##**Dataset 2 : Penn-Fudan Pedestrian**



In [ ]:
from train import main as train_main
train_main("configs/yolo_pennfudan.yaml")

In [ ]:
from evaluate import main as eval_main

eval_main(
    "configs/yolo_pennfudan.yaml",
    "outputs_pennfudan/train/weights/best.pt"
)

In [ ]:
from predict import main as pred_main

pred_main(
    "configs/yolo_pennfudan.yaml",
    "outputs_pennfudan/train/weights/best.pt",
    "datasets/PennFudanPed/images/val"
)

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# --- 1) Chercher tous les dossiers de prédictions YOLO ---
root = "runs/detect"

candidate_dirs = []
for d in os.listdir(root):
    full = os.path.join(root, d)
    if os.path.isdir(full):
        # On vérifie si le dossier contient des images prédictes
        imgs = [f for f in os.listdir(full) if f.lower().endswith((".jpg", ".png"))]
        if len(imgs) > 0:
            candidate_dirs.append(full)

# Si rien trouvé → essayer dans outputs_pennfudan
if len(candidate_dirs) == 0:
    alt = "outputs_pennfudan/pred/vis"
    if os.path.exists(alt):
        candidate_dirs = [alt]

# Sécurité
if len(candidate_dirs) == 0:
    raise ValueError("❌ Aucun dossier d'images prédictes trouvé.")

# --- 2) Prendre le plus récent ---
candidate_dirs = sorted(candidate_dirs)
last_pred = candidate_dirs[-1]
print("📂 Dossier de prédictions utilisé :", last_pred)

# --- 3) Lister les images ---
images = sorted([f for f in os.listdir(last_pred) if f.lower().endswith((".jpg", ".png"))])
print("🖼 Images trouvées :", images[:10])

# --- 4) Fonction d’affichage ---
def show_img(path):
    img = Image.open(path)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# --- 5) Afficher les 5 premières images ---
for img_name in images[:5]:
    print(f"\n➡️ Affichage : {img_name}")
    show_img(os.path.join(last_pred, img_name))